In [23]:
import numpy as np
import cv2

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

people = []
bl_people = []
face_coo = []


def find_faces_VJ(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.1, 5)
    return faces


def track_obj(faces, frame):
    track_windows = [(x, y, w, h) for (x, y, w, h) in faces]
    rois = [frame[y:y + h, x:x + w] for (x, y, w, h) in faces]
    hsv_rois = [cv2.cvtColor(roi, cv2.COLOR_BGR2HSV) for roi in rois]
    masks = [cv2.inRange(hsv_roi, np.array((0., 51., 89.)), np.array((17., 140., 255.))) for hsv_roi in hsv_rois]
    roi_hists = [cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180]) for (hsv_roi, mask) in zip(hsv_rois, masks)]
    for i in range(len(roi_hists)):
        cv2.normalize(roi_hists[i], roi_hists[i], 0, 255, cv2.NORM_MINMAX)
    term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

    return track_windows, rois, hsv_rois, masks, roi_hists, term_crit


def check_size(pts_prev, pts_now, size):
    return (cv2.contourArea(pts_now) / cv2.contourArea(pts_prev)) >= size


def intersection(pts1, pts2):
    # In future
    return True


def draw_face(frame, faces):
    for (x, y, w, h) in faces:
        print(1)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
    return 0

def flow(input_vid, output, frames_limit):
    cap = cv2.VideoCapture(input_vid)
    
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    ret, frame = cap.read()
    height, width, layers =  frame.shape
    out = cv2.VideoWriter(output, fourcc, 10.0, (width, height))
    current_frame = -1
    ret = True
    while(ret and current_frame < frames_limit):
        ret, frame = cap.read()
        current_frame += 1

#         if current_frame % 2 == 0: # each 5-th frame
        cur_faces = find_faces_VJ(frame)
        if current_frame == 0:
            faces = cur_faces
            for i in range(len(faces)):
                people.append([])
                face_coo.append([])
                bl_people.append(True)
            track_windows, rois, hsv_rois, masks, roi_hists, term_crit = track_obj(faces, frame)
        
        img2 = frame
#         out.write(img2)
        for i in range(len(people)):
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            dst = cv2.calcBackProject([hsv], [0], roi_hists[i], [0, 180], 1)
            
            
            print dst, track_windows[i], term_crit, "\n_____________"
            ret, track_windows[i] = cv2.CamShift(dst, track_windows[i], term_crit)
            
            
            pts = cv2.boxPoints(ret)

            pts = np.int0(pts)
            people[i] = pts
            face_coo[i].append(cv2.contourArea(pts))
            if face_coo[i][current_frame] / max(1,face_coo[i][current_frame if current_frame - 5 < 0 else current_frame - 5]) >= 2:
                bl_people[i] = False
                track_windows[i] = (track_windows[i][0]/2, track_windows[i][1]/2, track_windows[i][2]/2, track_windows[i][3]/2)
            img2 = cv2.polylines(frame, [pts], True, 255, 2)
            cv2.putText(img2, str(i), (pts[0][0], pts[0][1]), 1, 1, (0, 0, 255), 2, cv2.LINE_AA)

        out.write(img2)
#         print 1

    out.release()

flow('slice.mp4', 'output.avi', 10)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] (1031, 142, 26, 26) (3, 10, 1) 
_____________
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] (684, 132, 23, 23) (3, 10, 1) 
_____________
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] (748, 153, 24, 24) (3, 10, 1) 
_____________
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] (1111, 158, 26, 26) (3, 10, 1) 
_____________
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] (919, 203, 27, 27) (3, 10, 1) 
_____________
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] (544,

KeyboardInterrupt: 